## 1. Install Required Packages

In [ ]:
!pip install -qU langchain-openai langchain-community tavily-python python-dotenv langchain-core

## 2. Import Dependencies

In [ ]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Load environment variables
load_dotenv()

print("✓ Dependencies imported successfully")

✓ Dependencies imported successfully


## 3. Set Up API Keys

Required environment variables:
- `OPENAI_API_KEY` - Your OpenAI API key
- `TAVILY_API_KEY` - Your Tavily API key (get free at https://tavily.com)

In [ ]:
# Check API keys
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  OPENAI_API_KEY not found")
else:
    print("✓ OpenAI API key found")

if not os.getenv("TAVILY_API_KEY"):
    print("⚠️  TAVILY_API_KEY not found")
else:
    print("✓ Tavily API key found")

✓ OpenAI API key found
✓ Tavily API key found


## 4. Initialize LLM

In [ ]:
# Initialize the language model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

print("✓ LLM initialized")

✓ LLM initialized


## 5. Set Up Tavily Search Tool

In [ ]:
# Initialize Tavily search tool
search = TavilySearchResults(
    max_results=3,
    search_depth="advanced"
)

tools = [search]

print(f"✓ Tavily search tool initialized")
print(f"  Name: {search.name}")
print(f"  Description: {search.description}")

✓ Tavily search tool initialized
  Name: tavily_search_results_json
  Description: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.


## 6. Bind Tools to LLM

In [ ]:
# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

print("✓ Tools bound to LLM")

✓ Tools bound to LLM


In [29]:
# Install langgraph for the official agent
!pip install -qU langgraph

In [33]:
print("="*80)
print("Testing ReAct-Style Prompting with Chain")
print("="*80)

# Test the chain
response = react_chain.invoke({
    "question": "Who won the Nobel Prize in Physics in 2024?"
})

print("\n🧠 AGENT THINKING PROCESS:")
print("-" * 80)
print(response)
print("\n" + "="*80)

Testing ReAct-Style Prompting with Chain

🧠 AGENT THINKING PROCESS:
--------------------------------------------------------------------------------
Thought: The question is asking for information about the Nobel Prize in Physics for the year 2024, which is beyond my training data. I need to search for the most recent information regarding the Nobel Prize winners.  
Action: Search  
Action Input: "Nobel Prize in Physics 2024 winner"


🧠 AGENT THINKING PROCESS:
--------------------------------------------------------------------------------
Thought: The question is asking for information about the Nobel Prize in Physics for the year 2024, which is beyond my training data. I need to search for the most recent information regarding the Nobel Prize winners.  
Action: Search  
Action Input: "Nobel Prize in Physics 2024 winner"



## Test the Official ReAct Agent

In [34]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create a simpler ReAct-style chain with explicit prompting
react_template = """You are a helpful assistant with access to a search tool.

Answer the user's question using this EXACT format:

Thought: [Explain your reasoning about what you need to do]
Action: [Either "Search" or "Answer"]
Action Input: [If Search: your search query | If Answer: your final response]

Question: {question}

Let's begin:"""

react_prompt_template = PromptTemplate.from_template(react_template)

# Create a simple chain
react_chain = react_prompt_template | llm | StrOutputParser()

print("✓ ReAct-style chain created")
print("\nExample usage:")
print("response = react_chain.invoke({'question': 'your question here'})")

✓ ReAct-style chain created

Example usage:
response = react_chain.invoke({'question': 'your question here'})


In [35]:
response = react_chain.invoke({'question': "What's the difference between LangChain and LlamaIndex?"})

In [37]:
response

'Thought: I need to compare LangChain and LlamaIndex to highlight their differences, focusing on their functionalities, use cases, and any unique features they may have. \nAction: Search\nAction Input: "difference between LangChain and LlamaIndex"'

## 6b. Using LangChain's Built-in ReAct Agent

Let's use LangChain's official `create_react_agent` which shows chain of thought.

## 7. Create ReAct Agent with Chain of Thought

ReAct = Reasoning + Acting. The agent will explicitly think through each step.

In [26]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Create a ReAct prompt that forces the agent to show its thinking
react_system_prompt = """You are a helpful assistant that uses a search tool when needed.

For each query, you MUST follow this format to show your thinking process:

Thought: [Your reasoning about what you need to do]
Action: [The action you will take - either 'search' or 'respond']
Action Input: [If search: the query to search for. If respond: your final answer]

After getting search results:
Observation: [Summary of what you found]
Thought: [Your reasoning about the results]
Action: respond
Action Input: [Your final answer to the user]

IMPORTANT: Always show your Thought process before taking Action!"""

def run_react_agent(query: str):
    """Run a ReAct agent that shows explicit chain of thought reasoning."""
    
    print(f"\n{'='*80}")
    print(f"📥 USER QUERY: {query}")
    print(f"{'='*80}\n")
    
    # Start conversation with system prompt
    messages = [
        SystemMessage(content=react_system_prompt),
        HumanMessage(content=f"Question: {query}")
    ]
    
    max_iterations = 5
    for iteration in range(max_iterations):
        print(f"🔄 AGENT ITERATION {iteration + 1}")
        print("-" * 80)
        
        # Get agent's response
        response = llm.invoke(messages)
        agent_response = response.content
        
        print(f"🧠 AGENT OUTPUT:\n{agent_response}\n")
        
        # Parse the agent's response
        if "Action: search" in agent_response or "Action: Search" in agent_response:
            # Extract search query
            try:
                action_input_start = agent_response.find("Action Input:") + len("Action Input:")
                search_query = agent_response[action_input_start:].strip().split('\n')[0].strip()
                
                print(f"🔍 EXECUTING SEARCH...")
                print(f"   Query: {search_query}\n")
                
                # Perform search
                search_results = search.invoke({"query": search_query})
                
                print(f"📊 SEARCH RESULTS RECEIVED")
                print(f"   Found {len(search_results) if isinstance(search_results, list) else 1} result(s)\n")
                
                # Add results to conversation
                observation = f"Observation: Here are the search results:\n{search_results}\n\nNow provide your final answer."
                messages.append(AIMessage(content=agent_response))
                messages.append(HumanMessage(content=observation))
                
            except Exception as e:
                print(f"⚠️  Error parsing search: {e}")
                break
                
        elif "Action: respond" in agent_response or "Action: Respond" in agent_response:
            # Agent is ready to respond
            try:
                action_input_start = agent_response.find("Action Input:") + len("Action Input:")
                final_answer = agent_response[action_input_start:].strip()
                
                print(f"{'='*80}")
                print(f"✅ FINAL ANSWER:")
                print(f"{'='*80}")
                print(final_answer)
                print()
                
                return final_answer
            except:
                # If parsing fails, just return the response
                print(f"{'='*80}")
                print(f"✅ FINAL ANSWER:")
                print(f"{'='*80}")
                print(agent_response)
                print()
                return agent_response
        else:
            # No clear action detected, add to messages and continue
            messages.append(AIMessage(content=agent_response))
    
    print("⚠️  Max iterations reached")
    return "Could not complete the task within iteration limit."

print("✓ ReAct Agent ready with explicit Chain of Thought reasoning")

✓ ReAct Agent ready with explicit Chain of Thought reasoning


## 8. Example 1: Current Events

In [27]:
result = run_react_agent("What are the latest AI developments in December 2024?")


📥 USER QUERY: What are the latest AI developments in December 2024?

🔄 AGENT ITERATION 1
--------------------------------------------------------------------------------
🧠 AGENT OUTPUT:
Thought: Since my training only includes data up to October 2023, I need to search for the latest developments in AI specifically for December 2024 to provide accurate information. 
Action: search
Action Input: "latest AI developments December 2024"

🔍 EXECUTING SEARCH...
   Query: "latest AI developments December 2024"

🧠 AGENT OUTPUT:
Thought: Since my training only includes data up to October 2023, I need to search for the latest developments in AI specifically for December 2024 to provide accurate information. 
Action: search
Action Input: "latest AI developments December 2024"

🔍 EXECUTING SEARCH...
   Query: "latest AI developments December 2024"

📊 SEARCH RESULTS RECEIVED
   Found 3 result(s)

🔄 AGENT ITERATION 2
--------------------------------------------------------------------------------
📊 

## 9. Example 2: Comparison Query

In [ ]:
result = run_react_agent("What's the difference between LangChain and LlamaIndex?")


🤔 Processing query: What's the difference between LangChain and LlamaIndex?

📍 Iteration 1/5
--------------------------------------------------------------------------------
🔧 Tool calls detected: 3

  → Calling tool: tavily_search_results_json
  → Arguments: {'query': 'LangChain vs LlamaIndex'}
🔧 Tool calls detected: 3

  → Calling tool: tavily_search_results_json
  → Arguments: {'query': 'LangChain vs LlamaIndex'}
  → Result: [{'title': 'LangChain vs. LlamaIndex. Main differences - Addepto', 'url': 'https://addepto.com/blog/langchain-vs-llamaindex-main-differences/', 'content': 'Strategic Recommendation: LangChain ecosystem for complex, multi-agent production systems; LlamaIndex for document-centric, retrieval-focused applications [...] model. [...] and text classification.', 'score': 0.9999573}, {'title': 'LlamaIndex vs LangChain: Key Differences, Features & Use Cases', 'url': 'https://www.openxcell.com/blog/llamaindex-vs-langchain/', 'content': 'To Summarize: In the debate of Lang

## 10. Example 3: Factual Query

In [25]:
result = run_react_agent("Who won the Nobel Prize in Physics in 2024?")


📥 USER QUERY: Who won the Nobel Prize in Physics in 2024?

🔄 AGENT ITERATION 1
--------------------------------------------------------------------------------
🧠 AGENT OUTPUT:
Thought: The question is about the Nobel Prize in Physics for the year 2024, which is beyond my training data that goes up to October 2023. I need to search for the most recent information regarding the Nobel Prize winners for 2024. 
Action: search
Action Input: "Nobel Prize in Physics 2024 winner"

🔍 EXECUTING SEARCH...
   Query: "Nobel Prize in Physics 2024 winner"

🧠 AGENT OUTPUT:
Thought: The question is about the Nobel Prize in Physics for the year 2024, which is beyond my training data that goes up to October 2023. I need to search for the most recent information regarding the Nobel Prize winners for 2024. 
Action: search
Action Input: "Nobel Prize in Physics 2024 winner"

🔍 EXECUTING SEARCH...
   Query: "Nobel Prize in Physics 2024 winner"

📊 SEARCH RESULTS RECEIVED
   Found 3 result(s)

🔄 AGENT ITERATION

## Understanding ReAct Agent's Chain of Thought

The agent explicitly shows its reasoning using the **ReAct pattern** (Reasoning + Acting):

**Example thinking process:**
```
Thought: The user is asking about Nobel Prize winners in 2024. 
         I need to search for the most current information about this.
Action: search
Action Input: Nobel Prize in Physics 2024 winners

Observation: [Search results showing Geoffrey Hinton and John Hopfield won]

Thought: Based on the search results, I now have the information needed.
         I can provide a complete answer about the winners and their work.
Action: respond
Action Input: The 2024 Nobel Prize in Physics was awarded to...
```

This shows the **agent's internal reasoning** at each step - not just what tools it calls, but WHY it's doing what it's doing!

## Try Your Own Query

### Key Features of LangGraph ReAct Agent

**Advantages:**
- ✅ **Production-ready**: Built for real applications
- ✅ **Streaming support**: Real-time updates
- ✅ **Error handling**: Built-in retry logic
- ✅ **State management**: Maintains conversation history
- ✅ **Extensible**: Easy to add custom logic

**Agent Flow:**
1. Receives user input
2. **Thinks** about what to do
3. **Acts** by calling tools when needed
4. **Observes** the results
5. Repeats until it can answer
6. Returns final response

This is the **ReAct pattern** (Reasoning + Acting) in action!

In [ ]:
print("="*80)
print("🎯 LANGGRAPH REACT AGENT - ANOTHER EXAMPLE")
print("="*80)

query = "What are the latest developments in AI for December 2024?"
print(f"\n📥 Question: {query}\n")

# Simple invoke (non-streaming)
result = langgraph_agent.invoke(
    {"messages": [HumanMessage(content=query)]}
)

# Print the conversation
print("🔄 AGENT EXECUTION TRACE:\n")
for msg in result["messages"]:
    if msg.type == "human":
        print(f"👤 USER: {msg.content}\n")
    elif msg.type == "ai":
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            print("🤔 AGENT DECISION:")
            if msg.content:
                print(f"   {msg.content}")
            for tc in msg.tool_calls:
                print(f"   → Calling: {tc['name']}")
                print(f"   → Args: {tc['args']}\n")
        else:
            print("💬 AGENT RESPONSE:")
            print(f"   {msg.content}\n")
    elif msg.type == "tool":
        print(f"🔍 TOOL RESULT:")
        result_preview = str(msg.content)[:200] + "..." if len(str(msg.content)) > 200 else str(msg.content)
        print(f"   {result_preview}\n")

print("="*80)

### Another Example with Different Query

In [ ]:
print("="*80)
print("🎯 LANGGRAPH REACT AGENT - STREAMING EXAMPLE")
print("="*80)

query = "Who won the Nobel Prize in Physics in 2024?"
print(f"\n📥 Question: {query}\n")

# Stream the agent's execution
for step in langgraph_agent.stream(
    {"messages": [HumanMessage(content=query)]},
    stream_mode="values"
):
    # Get the last message in the step
    last_message = step["messages"][-1]
    
    # Pretty print based on message type
    if hasattr(last_message, 'type'):
        if last_message.type == "human":
            print(f"👤 USER: {last_message.content}\n")
        
        elif last_message.type == "ai":
            if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
                print("🤔 AGENT THINKING:")
                if last_message.content:
                    print(f"   Thought: {last_message.content}")
                
                for tool_call in last_message.tool_calls:
                    print(f"\n   🔧 Action: Call '{tool_call['name']}'")
                    print(f"   📝 Input: {tool_call['args']}")
                print()
            else:
                print("✅ FINAL ANSWER:")
                print(f"   {last_message.content}\n")
        
        elif last_message.type == "tool":
            print("📊 OBSERVATION:")
            # Truncate long results
            result = str(last_message.content)
            if len(result) > 300:
                result = result[:300] + "..."
            print(f"   {result}\n")

print("="*80)

### Run LangGraph Agent with Streaming

Stream the agent's thinking process in real-time:

In [ ]:
try:
    from langgraph.prebuilt import create_react_agent
    from langchain_core.messages import HumanMessage

    # Create the ReAct agent using LangGraph
    langgraph_agent = create_react_agent(llm, tools)

    print("✓ LangGraph ReAct Agent created successfully")
    print("\nThis agent will show its thinking process (ReAct pattern):")
    print("  - Thought: What the agent is thinking")
    print("  - Action: What tool to call")
    print("  - Observation: Results from the tool")
    print("  - Final Answer: The response to user")
    
except (ImportError, TypeError) as e:
    print(f"⚠️  LangGraph import error: {e}")
    print("\nAlternative: Using a custom ReAct agent implementation")
    print("The custom agent in section 7 works without LangGraph dependencies.")
    
    # Create a simple alternative
    class SimpleLangGraphAgent:
        def __init__(self, llm, tools):
            self.llm = llm
            self.tools = {tool.name: tool for tool in tools}
            
        def invoke(self, input_dict):
            from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
            messages = input_dict["messages"]
            all_messages = []
            
            for _ in range(5):  # max iterations
                response = self.llm.bind_tools(tools).invoke(messages)
                all_messages.append(response)
                messages.append(response)
                
                if not response.tool_calls:
                    return {"messages": all_messages}
                
                for tool_call in response.tool_calls:
                    tool_name = tool_call["name"]
                    if tool_name in self.tools:
                        result = self.tools[tool_name].invoke(tool_call["args"])
                        tool_msg = ToolMessage(
                            content=str(result),
                            tool_call_id=tool_call["id"]
                        )
                        all_messages.append(tool_msg)
                        messages.append(tool_msg)
            
            return {"messages": all_messages}
        
        def stream(self, input_dict, stream_mode="values"):
            result = self.invoke(input_dict)
            for i, msg in enumerate(result["messages"]):
                yield {"messages": result["messages"][:i+1]}
    
    langgraph_agent = SimpleLangGraphAgent(llm, tools)
    print("✓ Alternative ReAct agent created")

ImportError: cannot import name 'create_react_agent' from 'langgraph.prebuilt' (unknown location)

In [1]:
# Install LangGraph with compatible versions
!pip install -qU "langgraph>=0.2.0" "langchain-core>=0.3.0"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.25 requires langchain-core<0.3.0,>=0.2.40, but you have langchain-core 1.1.3 which is incompatible.
langchain-ollama 0.3.5 requires langchain-core<1.0.0,>=0.3.69, but you have langchain-core 1.1.3 which is incompatible.
langchain-text-splitters 0.2.4 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 1.1.3 which is incompatible.
langchain-community 0.2.19 requires langchain-core<0.3.0,>=0.2.43, but you have langchain-core 1.1.3 which is incompatible.
langchain-community 0.2.19 requires langsmith<0.2.0,>=0.1.112, but you have langsmith 0.4.58 which is incompatible.
langchain 0.2.17 requires langchain-core<0.3.0,>=0.2.43, but you have langchain-core 1.1.3 which is incompatible.
langchain 0.2.17 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.4.58 which is incompatibl

## LangGraph ReAct Agent Example

Using LangGraph's official `create_react_agent` for production-grade agents.

In [ ]:
# Your custom query here
my_query = "What's the weather like in San Francisco today?"

result = run_react_agent(my_query)